In [ ]:
import torch
import numpy as np
from lcmr.grammar import Scene
from lcmr.grammar.shapes.shape2d import Shape2D
from lcmr_ext.renderer.renderer2d import PyTorch3DRenderer2D
from lcmr.utils.presentation import display_img, make_img_grid
from lcmr.utils.colors import colors

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
raster_size = (128, 128)

In [ ]:
def single_shape_scene(fourierCoefficients, scale=[0.5, 0.5]):
    translation = torch.tensor([[0.5, 0.5]], dtype=torch.float32)[None, None, ...]
    color = torch.tensor([[0.2, 0.2, 0.2]], dtype=torch.float32)[None, None, ...]
    scale = torch.tensor([scale])[None, None, ...]
    confidence = torch.tensor([[0.9]])[None, None, ...]
    angle = torch.tensor([[0.0]], dtype=torch.float32)[None, None, ...]
    fourierCoefficients = torch.from_numpy(np.array([fourierCoefficients], dtype=np.float32))[None, None, ...]
    objectShape = torch.ones(size=(1, 1), dtype=torch.uint8)[None, None, ...] * Shape2D.FOURIER_SHAPE.value

    return Scene.from_tensors_sparse(
        translation=translation, scale=scale, color=color, confidence=confidence, angle=angle, fourierCoefficients=fourierCoefficients, objectShape=objectShape
    )

In [ ]:
renderer = PyTorch3DRenderer2D(raster_size, device=device, background_color=colors.white, n_verts=128)

In [ ]:
from pyefd import elliptic_fourier_descriptors

for order in [4, 8, 16, 32]:
    print("Order:", order)
    fourierCoefficients = elliptic_fourier_descriptors([[0, 0], [0, 1], [1, 1], [1, 0], [0, 0]], order=order)
    scene = single_shape_scene(fourierCoefficients)
    display_img(renderer.render(scene.to(renderer.device))[0])

In [ ]:
from math import sqrt

r = sqrt(2) / 2

fourierCoefficients = [[r, r, -r, r]] + [[0.0, 0.0, 0.0, 0.0]] * 7
scene = single_shape_scene(fourierCoefficients, scale=[0.25, 0.25]).to(device)

display_img(renderer.render(scene.to(renderer.device))[0])

In [ ]:
import imageio.v3 as iio
from torchvision.transforms.functional import resize


def resize_img(img):
    return resize(img.permute(0, 3, 1, 2), raster_size, antialias=True).permute(0, 2, 3, 1)


imgs = [
    "https://img.fruugo.com/product/2/60/557296602_max.jpg",
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS8Glk9GyCg-xOA1gGan8SM8TkbcaMli-7lQQ&usqp=CAU",
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSoZLVyXxYuVFO68LdjOhQ2Uxmqy0c68aFRG4L29xf93h7Sd8OXWmkgFAKf4Q0kfIvRfQU&usqp=CAU",
    "https://wheelsauto.com/media/catalog/product/cache/bdfbb51471fa4a0501abac6899ceb6a6/0/5/05094.jpg",
    "https://i.pinimg.com/736x/07/a5/8b/07a58be0fafc601e8bceef157ae01350.jpg",
    "https://static8.depositphotos.com/1338574/829/i/950/depositphotos_8292981-stock-photo-the-letter-y-in-gold.jpg",
    "https://i.etsystatic.com/5709149/r/il/da9a69/3283909570/il_570xN.3283909570_1lnw.jpg",
    "https://docs.telerik.com/devtools/winforms/telerik-presentation-framework/shapes/images/star-shape001.png"
    
]
imgs = [resize_img(torch.from_numpy(iio.imread(img)).to(torch.float32)[None, ...] / 255) for img in imgs]
imgs = torch.cat(imgs, dim=0)

display_img(make_img_grid([imgs]))

In [ ]:
from lcmr_ext.utils.optimize_params import optimize_params
from lcmr_ext.loss import LPIPSLoss

scene_opt = torch.cat([scene] * len(imgs)).to(device)
params = [
    scene_opt.layer.object.fourierCoefficients,
    scene_opt.layer.object.appearance.color,
    scene_opt.layer.object.transformation.translation,
    scene_opt.layer.object.transformation.angle,
    scene_opt.layer.object.transformation.scale,
]

optimize_params(scene_opt, imgs.to(device), renderer, params=params, show_progress=True, lr=0.001, epochs=2001, show_interval=200, Optimizer=torch.optim.Adam)